In [3]:
from pathlib import Path

import azureml
from IPython.display import display, Markdown
from azureml.core import Run, Model
from azureml.core import Datastore, Experiment, ScriptRunConfig, Workspace, RunConfiguration
from azureml.core.dataset import Dataset
from azureml.data import OutputFileDatasetConfig
from azureml.core.environment import Environment
from azureml.core.runconfig import DockerConfiguration
from azureml.exceptions import UserErrorException

from model_drift import settings, helpers
# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)


Azure ML SDK Version:  1.35.0


In [4]:
# Connect to workspace
ws = Workspace.from_config(settings.AZUREML_CONFIG)


In [10]:
input_dataset_name="results"

dbg = 0
experiment_name = 'common-runtime-test'
if dbg:
    experiment_name += "-dbg"
env_name = "vae"

exp = Experiment(workspace=ws, name=experiment_name)

use_commonrt = True
environment_file = settings.CONDA_ENVIRONMENT_FILE
project_dir = settings.SRC_DIR
pytorch_env = Environment.from_conda_specification(env_name, file_path =str(environment_file))
pytorch_env.register(workspace=ws)
build = pytorch_env.build(workspace=ws)
pytorch_env.environment_variables["RSLEX_DIRECT_VOLUME_MOUNT"] = "True"
pytorch_env.environment_variables["AZUREML_COMPUTE_USE_COMMON_RUNTIME"] = str(use_commonrt).lower()

# Run Configuration
run_config = RunConfiguration()
run_config.environment_variables["RSLEX_DIRECT_VOLUME_MOUNT"] = "True"
run_config.environment_variables["AZUREML_COMPUTE_USE_COMMON_RUNTIME"] = str(use_commonrt).lower()
run_config.environment = pytorch_env
run_config.docker = DockerConfiguration(use_docker=True, shm_size="100G")

dataset = Dataset.get_by_name(ws, name=input_dataset_name)

args = {
'run_azure': 1,
"input_dir": dataset.as_named_input('input').as_mount(),
}

config = ScriptRunConfig(
    source_directory=str(project_dir),
    script="scripts/azure-test.py",
    arguments=helpers.argsdict2list({}),
)
run_config.target = "cpu-cluster2"
config.run_config = run_config

run = exp.submit(config)
display(Markdown(f"""
- Experiment: [{run.experiment.name}]({run.experiment.get_portal_url()})
- Run: [{run.display_name}]({run.get_portal_url()})
- Target: {config.run_config.target}
"""))




- Experiment: [common-runtime-test](https://ml.azure.com/experiments/common-runtime-test?wsid=/subscriptions/9ca8df1a-bf40-49c6-a13f-66b72a85f43c/resourcegroups/MLOps-Prototype/workspaces/MLOps_shared&tid=72f988bf-86f1-41af-91ab-2d7cd011db47)
- Run: [boring_tangelo_c556mj74](https://ml.azure.com/runs/common-runtime-test_1643663987_8230b536?wsid=/subscriptions/9ca8df1a-bf40-49c6-a13f-66b72a85f43c/resourcegroups/MLOps-Prototype/workspaces/MLOps_shared&tid=72f988bf-86f1-41af-91ab-2d7cd011db47)
- Target: cpu-cluster2
